In [ ]:
!git clone https://github.com/openai/whisper.git

In [ ]:
!pip install torch openai-whisper

In [ ]:
# !pip install openai-whisper
import whisper
import os

def transcribe_audio(audio_path, model_name="base"):
    # Load the Whisper model
    model = whisper.load_model(model_name)

    # Perform the transcription
    result = model.transcribe(audio_path)

    # Extract the transcribed text
    transcribed_text = result["text"]

    return transcribed_text

# Example usage
audio_file_path = "conversation.wav"  # Replace with your audio file path
output_file_path = "transcription.txt"  # Path for the output text file

# Perform transcription
transcription = transcribe_audio(audio_file_path)

# Save transcription to a text file
with open(output_file_path, "w", encoding="utf-8") as file:
    file.write(transcription)

print(f"Transcription completed and saved to {output_file_path}")

In [ ]:
!pip install openai

In [ ]:
%cd /content


In [ ]:
import openai
import os
from typing import List, Tuple

# Set your OpenAI API key
openai.api_key = ""  # Replace with your actual API key

def read_file(file_path: str) -> str:
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

def split_into_chunks(text: str, chunk_size: int = 1000) -> List[str]:
    words = text.split()
    return [' '.join(words[i:i+chunk_size]) for i in range(0, len(words), chunk_size)]

def classify_text(chunk: str, prompt: str) -> str:
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that classifies text."},
            {"role": "user", "content": f"{prompt}\n\nText to classify:\n{chunk}"}
        ]
    )
    return response.choices[0].message['content'].strip()

def process_file(file_path: str, prompt: str) -> Tuple[List[str], List[str]]:
    text = read_file(file_path)
    chunks = split_into_chunks(text)

    label1_chunks = []
    label2_chunks = []

    for chunk in chunks:
        classification = classify_text(chunk, prompt)
        if "Label 1" in classification:
            label1_chunks.append(chunk)
        elif "Label 2" in classification:
            label2_chunks.append(chunk)

    return label1_chunks, label2_chunks

def save_results(label1_chunks: List[str], label2_chunks: List[str], output_dir: str):
    os.makedirs(output_dir, exist_ok=True)

    with open(os.path.join(output_dir, 'label1_text.txt'), 'w', encoding='utf-8') as f:
        f.write('\n'.join(label1_chunks))

    with open(os.path.join(output_dir, 'label2_text.txt'), 'w', encoding='utf-8') as f:
        f.write('\n'.join(label2_chunks))

# Main execution
if __name__ == "__main__":
    file_path = input("transcription.txt")
    prompt = input("please classify the female and male voice text and label them as male and female ")
    output_dir = input("opt.txt")

    label1_chunks, label2_chunks = process_file(file_path, prompt)
    save_results(label1_chunks, label2_chunks, output_dir)

    print(f"Classification complete. Results saved in {output_dir}")

In [ ]:
!pip install openai==0.28

In [ ]:
!pip install openai nltk

In [ ]:
import openai
import nltk
import os

# Download the punkt tokenizer for sentence splitting
nltk.download('punkt', quiet=True)

# Set your OpenAI API key
openai.api_key = "api key"  # Replace with your actual API key

def read_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

def split_into_sentences(text):
    return nltk.sent_tokenize(text)

def classify_sentence(sentence):
    prompt = f"Classify the following sentence as either doctor's speech or patient's speech. Respond with only 'Patient:' followed by the sentence.\n\nSentence: {sentence}"

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that classifies speech in medical conversations."},
            {"role": "user", "content": prompt}
        ]
    )

    return response.choices[0].message['content'].strip()

def process_transcription(input_file, output_file):
    # Read the transcription
    transcription = read_file(input_file)

    # Split into sentences
    sentences = split_into_sentences(transcription)

    # Classify each sentence
    classified_sentences = []
    for sentence in sentences:
        classification = classify_sentence(sentence)
        classified_sentences.append(classification)

    # Write the classified sentences to the output file
    with open(output_file, 'w', encoding='utf-8') as file:
        for sentence in classified_sentences:
            file.write(f"{sentence}\n")

# Main execution
if __name__ == "__main__":
    input_file = "transcription.txt"
    output_file = "opt.txt"

    process_transcription(input_file, output_file)

    print(f"Classification complete. Results saved in {output_file}")

In [ ]:
import re
import nltk

# Download the punkt tokenizer for sentence splitting
nltk.download('punkt', quiet=True)

# Lists of keywords and phrases often used by patients and doctors
patient_keywords = [
    "feel", "pain", "hurt", "worried", "concerned", "scared", "symptom",
    "problem", "issue", "medication", "medicine", "drug", "pill",
    "side effect", "family history", "allergy", "allergic"
]

doctor_keywords = [
    "diagnosis", "treatment", "prescription", "recommend", "suggest",
    "test", "examination", "results", "condition", "prognosis",
    "follow-up", "specialist", "referral", "dose", "medical history"
]

question_words = ["what", "how", "when", "where", "why", "do", "is", "are", "can", "could"]

def is_question(sentence):
    return any(sentence.lower().startswith(word) for word in question_words) or sentence.endswith("?")

def count_keywords(sentence, keyword_list):
    return sum(1 for keyword in keyword_list if keyword in sentence.lower())

def classify_sentence(sentence):
    patient_score = count_keywords(sentence, patient_keywords)
    doctor_score = count_keywords(sentence, doctor_keywords)

    # Doctors tend to ask more questions
    if is_question(sentence):
        doctor_score += 1

    # Patients often use first-person pronouns
    if re.search(r'\b(i|me|my|mine)\b', sentence.lower()):
        patient_score += 1

    # Doctors often use second-person pronouns
    if re.search(r'\b(you|your|yours)\b', sentence.lower()):
        doctor_score += 1

    if patient_score > doctor_score:
        return "Patient: " + sentence
    elif doctor_score > patient_score:
        return "Doctor: " + sentence
    else:
        return "Unclassified: " + sentence

def process_transcription(input_file, output_file):
    # Read the transcription
    with open(input_file, 'r', encoding='utf-8') as file:
        transcription = file.read()

    # Split into sentences
    sentences = nltk.sent_tokenize(transcription)

    # Classify each sentence
    classified_sentences = [classify_sentence(sentence) for sentence in sentences]

    # Write the classified sentences to the output file
    with open(output_file, 'w', encoding='utf-8') as file:
        for sentence in classified_sentences:
            file.write(f"{sentence}\n")

# Main execution
if __name__ == "__main__":
    input_file = "transcription.txt"
    output_file = "classified_conversation.txt"

    process_transcription(input_file, output_file)

    print(f"Classification complete. Results saved in {output_file}")

In [ ]:
!pip install transformers==2.2.0
!pip install bert-extractive-summarizer

In [ ]:
!pip install spacy==2.0.12


In [ ]:
from summarizer import Summarizer,TransformerSummarizer
body = '''
      Hello, how can I help you today? Hi, Doctor. I've been feeling very tired lately, and I seem to be getting hair make more frequently. How long have you been experiencing these symptoms? The tiredness has been going on for a few weeks, but the hair make started just a few days ago. Have you noticed any other symptoms, like changes in appetite or sleep patterns? Yes, actually. I've been having trouble sleeping, and I haven't been very hungry. I see. Have you been under a lot on stress recently? At work or home? Yes, work has been very stressful lately. It's possible that stress could be contributing to your symptoms. Have you tried any methods to reduce your stress, like exercise or relaxation techniques? I've tried, but I haven't been very consistent. I recommend trying to incorporate regular exercise and mindfulness practices into your routine. Additionally, let's run some blood tests to rule out any other possible causes. That sounds like a good plan. Thank you, Doctor. You're welcome. We'll schedule the tests, and follow up once we have the results. In the meantime, try to take it easy, and focus on self-care. I will. Thank you.

'''
bert_model = Summarizer()
bert_summary = ''.join(bert_model(body, min_length=60))
print(bert_summary)

In [ ]:
# !pip install transformers>=4.0.0 # Install a version of transformers that includes BartModel
# !pip install bert-extractive-summarizer
from transformers.models.bart.modeling_bart import BartModel
from summarizer import Summarizer, TransformerSummarizer
from typing import List, Optional, Union

from transformers import (AlbertModel, AlbertTokenizer, BigBirdModel, BigBirdTokenizer,
                          BertModel, BertTokenizer, CamembertModel, CamembertTokenizer, CTRLModel)
body = '''
      Hello, how can I help you today? Hi, Doctor. I've been feeling very tired lately, and I seem to be getting hair make more frequently. How long have you been experiencing these symptoms? The tiredness has been going on for a few weeks, but the hair make started just a few days ago. Have you noticed any other symptoms, like changes in appetite or sleep patterns? Yes, actually. I've been having trouble sleeping, and I haven't been very hungry. I see. Have you been under a lot on stress recently? At work or home? Yes, work has been very stressful lately. It's possible that stress could be contributing to your symptoms. Have you tried any methods to reduce your stress, like exercise or relaxation techniques? I've tried, but I haven't been very consistent. I recommend trying to incorporate regular exercise and mindfulness practices into your routine. Additionally, let's run some blood tests to rule out any other possible causes. That sounds like a good plan. Thank you, Doctor. You're welcome. We'll schedule the tests, and follow up once we have the results. In the meantime, try to take it easy, and focus on self-care. I will. Thank you.

'''
bert_model = Summarizer()
bert_summary = ''.join(bert_model(body, min_length=60))
print(bert_summary)

In [ ]:
!pip install --upgrade transformers

In [ ]:
!pip uninstall transformers summarizer
!pip install transformers summarizer

In [ ]:
!pip install --upgrade bert-extractive-summarizer

In [13]:
# Make sure you have the required libraries installed:
# !pip install transformers>=4.0.0
# !pip install bert-extractive-summarizer

# from summarizer import Summarizer, TransformerSummarizer
# from typing import List, Optional, Union

# from transformers import (AlbertModel, AlbertTokenizer, BigBirdModel, BigBirdTokenizer,
#                           BertModel, BertTokenizer, CamembertModel, CamembertTokenizer, CTRLModel)

body = '''
      Hello, how can I help you today? Hi, Doctor. I've been feeling very tired lately, and I seem to be getting hair make more frequently. How long have you been experiencing these symptoms? The tiredness has been going on for a few weeks, but the hair make started just a few days ago. Have you noticed any other symptoms, like changes in appetite or sleep patterns? Yes, actually. I've been having trouble sleeping, and I haven't been very hungry. I see. Have you been under a lot on stress recently? At work or home? Yes, work has been very stressful lately. It's possible that stress could be contributing to your symptoms. Have you tried any methods to reduce your stress, like exercise or relaxation techniques? I've tried, but I haven't been very consistent. I recommend trying to incorporate regular exercise and mindfulness practices into your routine. Additionally, let's run some blood tests to rule out any other possible causes. That sounds like a good plan. Thank you, Doctor. You're welcome. We'll schedule the tests, and follow up once we have the results. In the meantime, try to take it easy, and focus on self-care. I will. Thank you.
'''
from transformers import pipeline

summarizer = pipeline("summarization")
summary = summarizer(body, max_length=150, min_length=30, do_sample=False)

print(summary[0]['summary_text'])
# bert_model = Summarizer()
# bert_summary = ''.join(bert_model(body, min_length=60))
# print(bert_summary)

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


 Doctor: I've been feeling very tired lately, and I seem to be getting hair make more frequently . Have you noticed any other symptoms, like changes in appetite or sleep patterns? Have you been under a lot on stress recently? At work or home? It's possible that stress could be contributing to your symptoms .


In [ ]:
!pip install --upgrade transformers bert-extractive-summarizer spacy

In [14]:
!git clone https://huggingface.co/Clinical-AI-Apollo/Medical-NER

Cloning into 'Medical-NER'...
remote: Enumerating objects: 59, done.
remote: Counting objects: 100% (56/56), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 59 (delta 29), reused 0 (delta 0), pack-reused 3 (from 1)
Unpacking objects: 100% (59/59), 1.88 MiB | 1.93 MiB/s, done.


In [15]:
body1='''
Sure, here's a sample conversation between a doctor and a patient:

**Doctor:** Good morning! How can I help you today?

**Patient:** Good morning, Doctor. I’ve been feeling quite off lately, and I’m not sure what’s going on.

**Doctor:** I’m sorry to hear that. Can you describe what you mean by "feeling off"?

**Patient:** Well, I’ve been extremely tired all the time, even after getting a full night’s sleep. I also feel dizzy and lightheaded quite often, especially when I stand up quickly.

**Doctor:** That sounds concerning. Have you noticed any other symptoms?

**Patient:** Yes, I’ve also been having headaches almost daily. They’re not extremely painful, but they’re persistent and annoying. I’ve also lost my appetite and have been feeling nauseous on and off.

**Doctor:** I see. Have you experienced any changes in your weight, either gaining or losing, recently?

**Patient:** Yes, I’ve lost about 10 pounds in the last month without trying. It’s really strange because I haven’t changed my diet or exercise routine.

**Doctor:** That’s a significant amount of weight loss in a short period. How about your mood? Have you noticed any changes there?

**Patient:** Actually, yes. I’ve been feeling pretty down and anxious. I don’t have the same energy or interest in things I usually enjoy.

**Doctor:** I understand. How long have these symptoms been going on?

**Patient:** It’s been about two months now, and it seems to be getting worse.

**Doctor:** Thank you for sharing all of this. It’s important to get to the bottom of what’s causing these symptoms. I’d like to run some tests to check your blood work and see if there’s an underlying medical condition. It might also be helpful to discuss your mental health further, as sometimes physical symptoms can be linked to stress or anxiety.

**Patient:** That sounds like a good plan. I just want to feel better and understand what’s happening to my body.

**Doctor:** Absolutely, and we’ll do everything we can to figure this out. Let’s start with some blood tests and then schedule a follow-up appointment to go over the results. In the meantime, if your symptoms get worse or you have any new concerns, don’t hesitate to contact us.

**Patient:** Thank you, Doctor. I appreciate your help.

**Doctor:** You’re welcome. We’ll get to the bottom of this together. Take care, and I’ll see you soon.
'''
from transformers import pipeline

summarizer = pipeline("summarization")
summary = summarizer(body1, max_length=150, min_length=30, do_sample=False)

print(summary[0]['summary_text'])

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


 Patient: "I’ve been feeling quite off lately, and I’m not sure what’s going on" Patient: I've been extremely tired all the time, even after getting a full night's sleep . "I've also lost my appetite and have been feeling nauseous on and off," patient says .
